<a href="https://colab.research.google.com/github/sankardevisharath/amex-default-prediction/blob/master/notebooks/GRU_With_Complete_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from os import listdir

files = listdir('/content/drive/MyDrive/amex-default-prediction/data/customer_train/')

In [15]:
pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 13.8 MB/s 


In [16]:
import gc 

import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

import tensorflow as tf
from keras.layers import GRU, Dense
import tensorflow_addons as tfa


In [3]:
cat_cols = ['D_63', 'D_64','B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_66', 'D_68']
TRAIN_LABELS_PATH = '/content/train_labels.csv'

In [4]:
!unzip -o /content/drive/MyDrive/amex-default-prediction/data/amex-default-prediction.zip \
 train_labels.csv

Archive:  /content/drive/MyDrive/amex-default-prediction/data/amex-default-prediction.zip
  inflating: train_labels.csv        


In [5]:
train_labels = pd.read_csv(TRAIN_LABELS_PATH)

In [6]:
train_labels.shape

(458913, 2)

In [7]:
def append_label(source_df):
  return pd.merge(left=source_df, right=train_labels, how='inner')

In [8]:
def my_generator():
    for f in files:      
      X = pd.DataFrame()
      Y = pd.DataFrame()
      df_tmp = pd.read_parquet('/content/drive/MyDrive/amex-default-prediction/data/customer_train/' + f)
      df_tmp = append_label(df_tmp)
      enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
      df_tmp[['D_63', 'D_64']]=enc.fit(df_tmp[['D_63', 'D_64']])

      df_tmp[['D_63', 'D_64']]=enc.transform(df_tmp[['D_63', 'D_64']])
      df_tmp.fillna(0, inplace=True)

      X= X.append(df_tmp.loc[:, ~df_tmp.columns.isin(['target'])])
      X = X[X.groupby('customer_ID')['customer_ID'].transform('count') == 13]
      X_Cust = pd.DataFrame(X['customer_ID'].unique(), columns=['customer_ID'])
      Y = pd.merge(left=X_Cust, right=train_labels, how='inner')
      X.drop(axis=1, columns=['customer_ID', 'S_2'], inplace=True)
      X = X.values.reshape(X_Cust.shape[0], 13, 188)
      Y.drop(axis=1, columns='customer_ID', inplace=True)
      print(f'X shape = {X.shape}')
      yield X, Y.to_numpy()
      




In [9]:
def my_input_fn(epochs):
    dataset = tf.data.Dataset.from_generator(lambda: my_generator(),
                                             output_types=(tf.float64, tf.int64),
                                            output_shapes=(tf.TensorShape([None, 13, 188]), tf.TensorShape([None, 1])))

    dataset = dataset.repeat(epochs)
    # dataset = dataset.batch(32)
    return dataset


In [22]:
f1_metric = tfa.metrics.F1Score(num_classes=1, threshold=0.5)

model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(13, 188)))
model.add(GRU(64, dropout=0.1, recurrent_dropout=0.5))
model.add(Dense(6, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', f1_metric])



In [23]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_3 (GRU)                 (None, 64)                48768     
                                                                 
 dense_6 (Dense)             (None, 6)                 390       
                                                                 
 dense_7 (Dense)             (None, 1)                 7         
                                                                 
Total params: 49,165
Trainable params: 49,165
Non-trainable params: 0
_________________________________________________________________


In [24]:
epochs = 5
dataset = my_input_fn(epochs)
model.fit(dataset, epochs=epochs)


Epoch 1/5
X shape = (7508, 13, 188)
      1/Unknown - 7s 7s/step - loss: 0.9183 - accuracy: 0.2302 - f1_score: 0.3742X shape = (41994, 13, 188)
      2/Unknown - 23s 16s/step - loss: 0.8774 - accuracy: 0.2322 - f1_score: 0.3769X shape = (42037, 13, 188)
      3/Unknown - 43s 18s/step - loss: 0.8546 - accuracy: 0.2314 - f1_score: 0.3758X shape = (42038, 13, 188)
      4/Unknown - 61s 18s/step - loss: 0.8340 - accuracy: 0.2311 - f1_score: 0.3754X shape = (42098, 13, 188)
      5/Unknown - 79s 18s/step - loss: 0.8151 - accuracy: 0.2310 - f1_score: 0.3751X shape = (42029, 13, 188)
      6/Unknown - 99s 18s/step - loss: 0.7976 - accuracy: 0.2337 - f1_score: 0.3763X shape = (41911, 13, 188)
      7/Unknown - 117s 18s/step - loss: 0.7819 - accuracy: 0.2533 - f1_score: 0.3808X shape = (42089, 13, 188)
      8/Unknown - 136s 18s/step - loss: 0.7675 - accuracy: 0.3090 - f1_score: 0.3917X shape = (7508, 13, 188)
      9/Unknown - 139s 16s/step - loss: 0.7649 - accuracy: 0.3208 - f1_score: 0.3928X

In [25]:
val_files = listdir('/content/drive/MyDrive/amex-default-prediction/data/customer_val/')

In [26]:
def my_val_generator():
    for f in val_files:      
      X = pd.DataFrame()
      Y = pd.DataFrame()
      df_tmp = pd.read_parquet('/content/drive/MyDrive/amex-default-prediction/data/customer_val/' + f)
      df_tmp = append_label(df_tmp)
      enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
      df_tmp[['D_63', 'D_64']]=enc.fit(df_tmp[['D_63', 'D_64']])

      df_tmp[['D_63', 'D_64']]=enc.transform(df_tmp[['D_63', 'D_64']])
      df_tmp.fillna(0, inplace=True)

      X= X.append(df_tmp.loc[:, ~df_tmp.columns.isin(['target'])])
      X = X[X.groupby('customer_ID')['customer_ID'].transform('count') == 13]
      X_Cust = pd.DataFrame(X['customer_ID'].unique(), columns=['customer_ID'])
      Y = pd.merge(left=X_Cust, right=train_labels, how='inner')
      X.drop(axis=1, columns=['customer_ID', 'S_2'], inplace=True)
      X = X.values.reshape(X_Cust.shape[0], 13, 188)
      Y.drop(axis=1, columns='customer_ID', inplace=True)
      print(f'X shape = {X.shape}')
      yield X, Y.to_numpy()

In [27]:
def my_val_input_fn():
    dataset = tf.data.Dataset.from_generator(lambda: my_val_generator(),
                                             output_types=(tf.float64, tf.int64),
                                            output_shapes=(tf.TensorShape([None, 13, 188]), tf.TensorShape([None, 1])))

    # dataset = dataset.repeat(epochs)
    # dataset = dataset.batch(32)
    return dataset

In [28]:
val_dataset = my_val_input_fn()
model.evaluate(val_dataset)

X shape = (42194, 13, 188)
1/1 [==============================] - 15s 15s/step - loss: 0.3305 - accuracy: 0.9054 - f1_score: 0.7941


[0.33052611351013184, 0.9054368138313293, array([0.7940964], dtype=float32)]

In [ ]:
y_pred = (model.predict(val_dataset) > 0.5).astype("int32")

In [ ]:
f1_score(val_dataset[1], y_pred)